<a href="https://colab.research.google.com/github/JonasHell/aml_monet/blob/main/Monet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
%cd /content/gdrive/MyDrive/CINN

/content/gdrive/MyDrive/CINN


In [ ]:
!git clone https://github.com/JonasHell/aml_monet.git

Cloning into 'aml_monet'...
fatal: could not read Username for 'https://github.com': No such device or address


In [ ]:
pip install git+https://github.com/VLL-HD/FrEIA.git

  Cloning https://github.com/VLL-HD/FrEIA.git to /tmp/pip-req-build-uvlcukas
  Running command git clone -q https://github.com/VLL-HD/FrEIA.git /tmp/pip-req-build-uvlcukas
     |████████████████████████████████| 28.5 MB 84 kB/s 
  Created wheel for FrEIA: filename=FrEIA-0.2-py3-none-any.whl size=48967 sha256=4169641c358bb375c7e7e515de971b69b1c0edd61bea85ae470316dff30ee168
  Stored in directory: /tmp/pip-ephem-wheel-cache-lorzqzei/wheels/ce/20/ea/82f4b732f0705014bd0b340c3375d25aa89491a98a49ec4b8f
Successfully built FrEIA
  Attempting uninstall: scipy
    Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [ ]:
#################
# Architecture: #
#################
img_width  = 112
img_height = 112

#############################
# Training hyperparameters: #
#############################
N_epochs    = 10
lr          = 1e-3
batch_size  = 16

#Total number of images used for training
N_train     = 512

#We take the first N_test images from the test dataset for validation and the N_val images after that for validation
N_test      = 128
N_val       = 64

#######################
# Dataset parameters: #
#######################

#Root folder for dataset
data_root = '/content/gdrive/MyDrive/Daniel-Daten/'

#The code expects the images in the training and test folders to be named real1.jpg, real2.jpg, ..., fake1.jpg, fake2.jpg, ...
training_img_folder    =  data_root + 'cyclegan train/fake/'
training_cond_folder   =  data_root + 'cyclegan train/real/'
test_img_folder        =  data_root + 'cyclegan test/fake/' 
test_cond_folder       =  data_root + 'cyclegan test/real/'

########################
# Display and logging: #
########################

output_root = '/content/gdrive/MyDrive/CINN/'
output_image_folder     = output_root + 'images'

#######################
# Saving checkpoints: #
#######################

#This is where the training algorithm stores the model
model_output = output_root + 'monet_cinn.pt'
#This is where the evaluation algorithm reas the model from
model_path   = model_output

In [ ]:
import numpy as np
from skimage import io, color
import torch
from PIL import Image
from torch.utils.data import Dataset, DataLoader, TensorDataset
import torchvision.transforms as T
import albumentations

"""
Implement dataset for pytorch dataloader that returns source images as well as condition images
and create training and test dataloaders.

Remember to:
- Set correct paths to images
- Adapt image size and data augmentation to your needs
"""

"""
Custom dataset for holding photo + artistic rendering of photo pairs of RGB images
Supports:
-   cropping images to desired size (128 pixels x 128 pixels by default)
-   normalising images
-   data augmentation
-   adding noise to training data

Returns:
-   Tuples of source images with their respective condition images
"""

class PairDataset(Dataset):

  #Resize dataset, normalise data and augment if is_valid = 0
  def __init__(self, image_paths, condition_paths,  transform = True, noise = False, img_height = img_height, img_width = img_width, mean = (0, 0, 0), std  = (1, 1, 1)):
    self.image_paths        = image_paths
    self.condition_paths    = condition_paths
    #Check whether there is the same number of images and condition
    assert(len(image_paths) == len(condition_paths))
    self.img_height         = img_height
    self.img_width          = img_width
    self.transform          = transform
    self.noise              = noise


    if self.transform == False:
      self.aug = albumentations.Compose([
                                         albumentations.CenterCrop(img_height, img_width, always_apply = True),
                                         albumentations.Normalize(mean, std, always_apply = True)
      ])
    else:
      #Apply affine transformations to scale, shift and rotate input images
      self.aug = albumentations.Compose([
                                         albumentations.RandomCrop(img_height, img_width, always_apply = True),
                                         albumentations.Normalize(mean, std, always_apply = True),
                                         albumentations.ShiftScaleRotate(shift_limit = 0.0625, 
                                                                         scale_limit = 0.1, 
                                                                         rotate_limit = 5,
                                                                         p = 0.9)
      ])

  def __len__(self):
    return len(self.image_paths)

  def get_image(self, filename):
    #Open image and convert to numpy array 
    img = np.array(Image.open(filename))
    
    #Resize to desired size and interpolate if necessary
    #Normalise and apply data augmentation
    #img = cv2.resize(img, dsize=(self.img_width, self.img_height), interpolation = cv2.INTER_CUBIC)
    img = self.aug(image = img)["image"]

    #Convert back to pytorch data structure
    img = np.transpose(img, (2,0,1)).astype(np.float32) # changing format s.t. pytorch will accept it
    img = torch.tensor(img, dtype = torch.float)

    #Add noise to improve training performance
    if self.noise:
        img += 0.005 * torch.rand_like(img)

    return img

  #Return tuple of image and its condition (another image)
  def __getitem__(self, index):
    image = self.get_image(self.image_paths[index])
    condition = self.get_image(self.condition_paths[index])
    return image, condition



training_img_list   =  [training_img_folder  + f'fake{i}.jpg' for i in range(1, 1 + N_train)]
training_cond_list  =  [training_cond_folder + f'real{i}.jpg' for i in range(1, 1 + N_train)]
test_img_list       =  [test_img_folder      + f'fake{i}.jpg' for i in range(1, 1 + N_test)]
test_cond_list      =  [test_cond_folder     + f'real{i}.jpg' for i in range(1, 1 + N_test)]
val_img_list        =  [test_img_folder      + f'fake{i}.jpg' for i in range(2 + N_test, 2 + N_test + N_val)]
val_cond_list       =  [test_cond_folder     + f'real{i}.jpg' for i in range(2 + N_test, 2 + N_test + N_val)]

train_data = PairDataset(training_img_list, training_cond_list, transform=True, noise=True)
test_data  = PairDataset(test_img_list, test_cond_list        , transform=False, noise=False)
val_data  =  PairDataset(val_img_list, val_cond_list          , transform=False, noise=False)


train_loader = DataLoader(train_data,   batch_size=batch_size, shuffle=True,    num_workers=8,  pin_memory=True, drop_last=True)
test_loader  = DataLoader(test_data,    batch_size=batch_size, shuffle=False,   num_workers=4,  pin_memory=True, drop_last=False)

#Load all test and validation images and append them to a list
#stack concatenates a sequence of tensors along a new dimension
#list creates a list using the __get_item__ function
x  = list(test_data)
tx = list(zip(*x))
test_img_all  = torch.stack(tx[0], 0).cuda()
test_cond_all  = torch.stack(tx[1], 0).cuda()

x  = list(val_data)
tx = list(zip(*x))
val_img_all  = torch.stack(tx[0], 0).cuda()
val_cond_all  = torch.stack(tx[1], 0).cuda()
"""
Mit den folgenden Zeilen kann man den Code testen und sich den Effekt der Data Augmentation für das Trainingsdatenset anschauen

import matplotlib.pyplot as plt
%matplotlib inline
id = 64
img1, img2 = test_data[id]
plt.imshow(np.transpose(img1.numpy(), (1, 2, 0)))
plt.imshow(np.transpose(img2.numpy(), (1, 2, 0)))


"""

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


'\nMit den folgenden Zeilen kann man den Code testen und sich den Effekt der Data Augmentation für das Trainingsdatenset anschauen\n\nimport matplotlib.pyplot as plt\n%matplotlib inline\nid = 64\nimg1, img2 = test_data[id]\nplt.imshow(np.transpose(img1.numpy(), (1, 2, 0)))\nplt.imshow(np.transpose(img2.numpy(), (1, 2, 0)))\n\n\n'

In [49]:
# %%
import torch
import torch.nn as nn
from torch.nn.modules.activation import ReLU
import torch.optim

import FrEIA.framework as Ff
import FrEIA.modules as Fm

# %%
ndim_total = 3*img_width*img_height

class ConditionNet(nn.Module):
    def __init__(self):
        super().__init__()

        self.res_blocks = nn.ModuleList([
            nn.Sequential(
                # 3 x 224 x 224
                nn.Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
                # 64 x 224 x 224
                nn.ReLU(inplace=True),
                nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
                # 64 x 112 x 112
                nn.Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
                # 128 x 112 x 112
            ),
            nn.Sequential(
                nn.ReLU(inplace=True),
                nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
                # 128 x 56 x 56
                nn.Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
                # 256 x 56 x 56
                nn.ReLU(inplace=True),
                nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
                # 256 x 56 x 56
            ),
            nn.Sequential(
                nn.ReLU(inplace=True),
                nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
                # 256 x 28 x 28
                nn.Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
                # 512 x 28 x 28
                nn.ReLU(inplace=True),
                nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
                # 512 x 28 x 28
            ),
            nn.Sequential(
                nn.ReLU(inplace=True),
                nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
                # 512 x 14 x 14
                nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
                # 512 x 14 x 14
                nn.ReLU(inplace=True),
                nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
                # 512 x 14 x 14
            ),
            nn.Sequential(
                nn.ReLU(inplace=True),
                nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
                # 512 x 7 x 7
            ),
            nn.Sequential(
                nn.AdaptiveAvgPool2d(output_size=(7, 7)),
                # 25088
                nn.Linear(in_features=25088, out_features=4096, bias=True),
                # 4096
                nn.ReLU(inplace=True),
                nn.Dropout(p=0.5, inplace=False),
                nn.Linear(in_features=4096, out_features=4096, bias=True),
                # 4096
                #nn.ReLU(inplace=True),
                #nn.Dropout(p=0.5, inplace=False), # TODO: with dropout?
                #nn.Linear(in_features=4096, out_features=1000, bias=True)
                # 1000
            )
        ])
        
    def forward(self, photo):
        outputs = [photo]
        for module in self.res_blocks:
            outputs.append(module(outputs[-1]))
        return outputs[1:]

    # evtl auch möglich:
    # https://pytorch.org/tutorials/beginner/finetuning_torchvision_models_tutorial.html
    # dann einfach in forward die entsprechenden unter module aufrufen, vermutlich baer ähnlich großer aufwand
        

class MonetCINN_112_blocks10(nn.Module):
    def __init__(self, learning_rate):
        super().__init__()

        self.cinn = self.create_cinn()
        self.cond_net = ConditionNet()

        self.trainable_parameters = [p for p in self.cinn.parameters() if p.requires_grad]
        for p in self.trainable_parameters:
            p.data = 0.02 * torch.randn_like(p)

        self.trainable_parameters += list(self.cond_net.parameters())
        self.optimizer = torch.optim.Adam(self.trainable_parameters, lr=learning_rate)

    def create_cinn(self):
    
        def subnet_conv(hidden_channels_1, hidden_channels_2, kernel_size):
            padding = kernel_size // 2
            return lambda in_channels, out_channels: nn.Sequential(
                nn.Conv2d(in_channels, hidden_channels_1, kernel_size, padding=padding),
                nn.ReLU(),
                nn.Conv2d(hidden_channels_1, hidden_channels_2, kernel_size, padding=padding),
                nn.ReLU(),
                nn.BatchNorm2d(hidden_channels_2),
                nn.Conv2d(hidden_channels_2, out_channels, kernel_size, padding=padding)
            )

        def subnet_fc(hidden_channels_1, hidden_channels_2):
            return lambda in_channels, out_channels: nn.Sequential(
                nn.Linear(in_channels, hidden_channels_1),
                nn.ReLU(),
                nn.Linear(hidden_channels_1, hidden_channels_2),
                nn.ReLU(),
                nn.Linear(hidden_channels_2, out_channels)
            )

        def add_stage(nodes, block_num, subnet_func, condition=None, split_nodes=None, split_sizes=None, downsample=True, prefix=''):
            """
            Convenience function that adds an entire stage to nodes.
            """
            #TODO: does appending work correctly?

            # add specified number of blocks
            for k in range(block_num):
                subnet = subnet_func(block_num)
                
                # add current block
                nodes.append(Ff.Node(
                    nodes[-1],
                    Fm.GLOWCouplingBlock,
                    {'subnet_constructor': subnet, 'clamp': 2.0},
                    conditions=condition, #TODO
                    name=prefix+f'-block{k+1}'
                ))

                # add permutation after each block
                nodes.append(Ff.Node(
                    nodes[-1],
                    Fm.PermuteRandom,
                    {},
                    name=prefix+f'-block{k+1}-perm'
                ))
            #print(nodes[-1])
            # split channels off
            if split_nodes is not None:
                nodes.append(Ff.Node(
                    nodes[-1],
                    Fm.Split,
                    {'section_sizes': split_sizes, 'dim': 0},
                    name=prefix+'split'
                ))
                split_nodes.append(Ff.Node(
                    nodes[-1].out1,
                    Fm.Flatten,
                    {},
                    name=prefix+'flatten'
                ))

            # add downsampling at the end of stage
            if downsample:
                nodes.append(Ff.Node(
                    nodes[-1],
                    Fm.HaarDownsampling,
                    {'rebalance': 0.5},
                    name=prefix+'-down'
                ))
            
        # create nodes with input node
        #nodes = [Ff.InputNode(3, 256, 256)]
        nodes = [Ff.InputNode(3, 112, 112)]

        # create conditions
        condition_nodes = [ Ff.ConditionNode(128, 112, 112),
                            Ff.ConditionNode(256, 56, 56),
                            Ff.ConditionNode(512, 28, 28),
                            Ff.ConditionNode(512, 14, 14),
                            Ff.ConditionNode(512, 7, 7),
                            Ff.ConditionNode(4096)] #TODO: 1000 or 4096?

        # create split_nodes
        split_nodes = []
        
        # stage 1
        # one block (3 x 112 x 112)
        # with conv3 subnet
        subnet_func = lambda _: subnet_conv(32, 64, 3)
        add_stage(nodes, 1, subnet_func,
            condition=condition_nodes[0],
            prefix='stage1'
        )

        # stage 2
        # two blocks (12 x 56 x 56)
        # one with conv1 and one with conv3 subnet
        subnet_func = lambda block_num: subnet_conv(64, 128, 3 if block_num%2 else 1)
        add_stage(nodes, 2, subnet_func,
            condition=condition_nodes[1],
            split_nodes=split_nodes,
            prefix='stage2'
        )

        # stage 3
        # two blocks (24 x 28 x 28)
        # one with conv1 and one with conv3 subnet
        subnet_func = lambda block_num: subnet_conv(128, 256, 3 if block_num%2 else 1)
        add_stage(nodes, 2, subnet_func,
            condition=condition_nodes[2],
            split_nodes=split_nodes,
            prefix='stage3'
        )

        # stage 4
        # two blocks (48 x 14 x 14)
        # one with conv1 and one with conv3 subnet
        subnet_func = lambda block_num: subnet_conv(128, 256, 3 if block_num%2 else 1)
        add_stage(nodes, 2, subnet_func,
            condition=condition_nodes[3],
            split_nodes=split_nodes,
            prefix='stage4'
        )
        #TODO: does it make sense to increase num of channels in subnets?
        #TODO: should they be larger in the beginning due to condition
        #print(nodes[-1])
        # stage 5
        # two blocks (96 x 7 x 7)
        # one with conv1 and one with conv3 subnet
        subnet_func = lambda block_num: subnet_conv(128, 256, 3 if block_num%2 else 1)
        add_stage(nodes, 2, subnet_func,
            condition=condition_nodes[4],
            downsample=False,
            split_nodes=split_nodes,
            split_sizes=[24, 72],
            prefix='stage5'
        )

        # flatten for fc part
        nodes.append(Ff.Node(
            nodes[-1],
            Fm.Flatten,
            {},
            name='flatten'
        ))

        # stage 6
        # one block (flat 1176)
        # with fc subnetwork
        subnet_func = lambda _: subnet_fc(1024, 1024)
        add_stage(nodes, 1, subnet_func,
            condition=condition_nodes[5],
            downsample=False,
            prefix='stage6'
        )
        #print(nodes[-1])
        # concat all the splits and the output of fc part
        nodes.append(Ff.Node(
            [sn.out0 for sn in split_nodes] + [nodes[-1].out0],
            Fm.Concat,
            {'dim':0},
            name='concat'
        ))
        #print(nodes[-1])
        # add output node
        nodes.append(Ff.OutputNode(nodes[-1], name='output'))
        #print(nodes[-1])
        #TODO: use GraphINN or ReversibleGraphNet??
        return Ff.ReversibleGraphNet(nodes + split_nodes + condition_nodes)

    def forward(self, monet, photo):
        z = self.cinn(monet, c=self.cond_net(photo))
        jac = self.cinn.log_jacobian(run_forward=False)
        return z, jac

    def reverse_sample(self, z, photo):
        return self.cinn(z, c=self.cond_net(photo), rev=True)

        # am ende fc? oder muss conv dann iwie so passen dass am ende was sinvolles rauskommt
        # muss latent space gleiche dim wie original space haben
        # welche dim müssen conditions haben
        # wieviel channel in subnet?

In [50]:
from time import time

#from tqdm import tqdm
import torch
import torch.optim
import numpy as np


"""
Training routine for INN
Remember to:
- Adapt number of epochs
- Set learning rate
- Set output directory for model
"""

cinn = MonetCINN_112_blocks10(lr)
cinn.cuda()
scheduler = torch.optim.lr_scheduler.StepLR(cinn.optimizer, 1, gamma=0.1)

N_epochs = N_epochs
t_start = time()
nll_mean = []

print('Epoch\tBatch/Total \tTime \tNLL train\tNLL val\tLR')
for epoch in range(N_epochs):
    for i, images in enumerate(train_loader):
        #train_loader returns a list with two elements
        #Both elements are torch tensors with size (batch_size x 3 (RGB channels) x image width x image height)
        #We immediately load every batch of source and condition images to the GPU
        source    = images[0].cuda()
        condition = images[1].cuda()

        #We pass both the source image as well as the condition image to the INN
        """
        This needs to be adapted depending on the final architecture of the INN
        """
        z, log_j = cinn(source, condition)

        #Compute the loss of the INN
        nll = torch.mean(z**2) / 2 - torch.mean(log_j) / ndim_total
        nll.backward()
        nll_mean.append(nll.item())
        cinn.optimizer.step()
        cinn.optimizer.zero_grad()

        #Print training progress every 20 batches
        if not i % 20:
            #Compute training loss for validation images
            with torch.no_grad():
                """
                This needs to be adapted depending on the final architecture of the INN
                """
                z, log_j = cinn(val_img_all, val_cond_all)
                nll_val = torch.mean(z**2) / 2 - torch.mean(log_j) / ndim_total

            print('%.3i \t%.5i/%.5i \t%.2f \t%.6f\t%.6f\t%.2e' % (epoch,
                                                            i, len(train_loader),
                                                            (time() - t_start)/60.,
                                                            np.mean(nll_mean),
                                                            nll_val.item(),
                                                            cinn.optimizer.param_groups[0]['lr'],
                                                            ), flush=True)
            nll_mean = []

    scheduler.step()
torch.save(cinn.state_dict(), model_output)

# %%


ReversibleGraphNet(
  (module_list): ModuleList(
    (0): GLOWCouplingBlock(
      (subnet1): Sequential(
        (0): Conv2d(129, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): ReLU()
        (2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (3): ReLU()
        (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (5): Conv2d(64, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      )
      (subnet2): Sequential(
        (0): Conv2d(130, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): ReLU()
        (2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (3): ReLU()
        (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (5): Conv2d(64, 2, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      )
    )
    (1): PermuteRandom()
    (2): HaarDownsampling()
    (3): GLOWCouplingBlock(
      (subnet1): Seque

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


RuntimeError: ignored